In [1]:
from datetime import datetime
import pytz

def fecha_hora_lima():
    lima_tz = pytz.timezone('America/Lima')
    lima_time = datetime.now(lima_tz)
    return lima_time.strftime('%Y%m%d-%H%M%S')

def fecha_hora_lima_db():
    lima_tz = pytz.timezone('America/Lima')
    lima_time = datetime.now(lima_tz)
    return lima_time.strftime('%Y-%m-%d %H:%M:%S')

fecha_hora_para_archivos = fecha_hora_lima()
fecha_hora_para_base_de_datos = fecha_hora_lima_db()

In [2]:
import pandas as pd
import requests
import json

# URL del JSON
url = 'https://www.metro.pe/api/catalog_system/pub/category/tree/3'

# Solicitar el contenido JSON desde la URL
response = requests.get(url)

# Verificar si la solicitud fue exitosa
if response.status_code == 200:
    data = response.json()  # Convertir la respuesta en un objeto JSON
    file_name = f'categories_{fecha_hora_para_archivos}.json'
    
    # Guardar el JSON en un archivo
    with open(file_name, 'w') as file:
        json.dump(data, file, indent=4)

    print('El archivo JSON se ha guardado exitosamente.')
    with open(file_name, 'r') as file:
        data = json.load(file)
    print(len(data)) 
else:
    print(f'Error al obtener el JSON. Código de estado: {response.status_code}')  

El archivo JSON se ha guardado exitosamente.
26


In [3]:
# file_path = 'metro_categories.json'
# 
# with open(file_path, 'r') as file:
#     data = json.load(file)
#     print(json.dumps(data, indent=4))

In [4]:
columns = ['id', 'name', 'hasChildren', 'url', 'NumChildren', 'Title', 'MetaTagDescription', 'parent_id']
df = pd.DataFrame(columns=columns)
df

,id,name,hasChildren,url,NumChildren,Title,MetaTagDescription,parent_id


In [5]:
for item in data:
  if item['hasChildren']:
    item['parent_id'] = ''
    item['NumChildren'] = len(item['children'])

    df.loc[len(df)] = item

    parent_id = item['id']

    for subitem in item['children']:
      subitem['parent_id'] = parent_id
      sub_parent_id = subitem['id']
      subitem['NumChildren'] = len(subitem['children'])

      df.loc[len(df)] = subitem

      if len(subitem['children']) > 0:
        for subsubitem in subitem['children']:
          subsubitem['parent_id'] = sub_parent_id
          subsubitem['NumChildren'] = len(subsubitem['children'])
          df.loc[len(df)] = subsubitem

In [6]:
df.query("id == 812")


,id,name,hasChildren,url,NumChildren,Title,MetaTagDescription,parent_id
4,812,Fresas y Otros Berries,False,https://www.metro.pe/frutas-y-verduras/frutas/...,0,"Fresas y Otros Frutos Rojos - Arándanos, Moras...",Descubre una variedad de fresas y frutos rojos...,801


In [7]:
file_name_categories_csv = f'metro_categories_{fecha_hora_para_archivos}.csv'
df.to_csv(file_name_categories_csv, index=False)

In [8]:
cats = df.query("parent_id == ''")['id'].to_list()
print(cats)
print(len(cats))

[800, 1200, 1400, 1700, 1800, 1900, 2000, 2100, 2200, 2301, 1000003, 1000069, 1000131, 1000143, 1000144, 1001074, 1001253, 1001327, 1001374, 1001402, 1001436, 1001467, 1001470, 1001701]
24


In [9]:
import time

products = []

for cat in cats:
    start = 0
    end = 49
    
    while True:
        time.sleep(1)
        url = f"https://www.metro.pe/api/catalog_system/pub/products/search?fq=C:{cat}&_from={start}&_to={end}&O=OrderByPriceDESC"
        
        response = requests.get(url)
        
        if response.status_code == 200 or response.status_code == 206:
            data = response.json()
            if not data:
                break
                 
            products.extend(data)
            start += 50
            end += 50
        else:
            break
    
    print(f"status_code = {response.status_code}")
    print(f"cat = {cat}, start = {start}, end = {end}")

 
file_name_productos_csv = f'productos_{fecha_hora_para_archivos}.json'

with open(file_name_productos_csv, 'w') as file:
    json.dump(products, file, indent=4)

status_code = 200
cat = 800, start = 500, end = 549
status_code = 200
cat = 1200, start = 400, end = 449
status_code = 200
cat = 1400, start = 500, end = 549
status_code = 500
cat = 1700, start = 1100, end = 1149
status_code = 200
cat = 1800, start = 600, end = 649
status_code = 200
cat = 1900, start = 1150, end = 1199
status_code = 200
cat = 2000, start = 900, end = 949
status_code = 200
cat = 2100, start = 1350, end = 1399
status_code = 200
cat = 2200, start = 2350, end = 2399
status_code = 200
cat = 2301, start = 1850, end = 1899
status_code = 200
cat = 1000003, start = 400, end = 449
status_code = 200
cat = 1000069, start = 350, end = 399
status_code = 400
cat = 1000131, start = 2550, end = 2599
status_code = 200
cat = 1000143, start = 850, end = 899
status_code = 200
cat = 1000144, start = 300, end = 349
status_code = 200
cat = 1001074, start = 1500, end = 1549
status_code = 200
cat = 1001253, start = 1150, end = 1199
status_code = 200
cat = 1001327, start = 550, end = 599
status_

In [10]:
print(len(products))

20594


In [11]:
columns = ['productId', 'productName', 'brand', 'brandId', 'brandImageUrl','categoryId',
           'productTitle', 'metaTagDescription', 'categories', 'categoriesIds', 'link', 'Vendido por',
           'Descripción del producto', 'Presentación', 'Presentación/Empaque', 'Pack', 'Porción Sugerida',
           'Azúcares Por Porción', 'Carbohidratos Por Porción', 'Descripción Nutricional', 'Grasas Por Porción',
           'Advertencias de consumo', 'itemId', 'ean', 'imageUrl', 'Value', 'IsAvailable', 'AvailableQuantity',
           'Price', 'ListPrice', 'PriceWithoutDiscount', 'PriceValidUntil', 'Created']

df = pd.DataFrame(columns=columns)
df

,productId,productName,brand,brandId,brandImageUrl,categoryId,productTitle,metaTagDescription,categories,categoriesIds,...,ean,imageUrl,Value,IsAvailable,AvailableQuantity,Price,ListPrice,PriceWithoutDiscount,PriceValidUntil,Created


In [12]:
for product in products:
  print(product['items'][0]['sellers'][0]['commertialOffer']['Installments'][0]['Value'])
  print(product['items'][0]['sellers'][0]['commertialOffer']['PriceValidUntil'])
  print(product['items'][0]['sellers'][0]['commertialOffer']['ListPrice'])
  print(product['items'][0]['sellers'][0]['commertialOffer']['PriceWithoutDiscount'])
  break

189.0
2100-01-01T00:00:00Z
189.0
189.0


In [13]:
print(json.dumps(products[3099], indent=4))

{
    "productId": "1017279",
    "productName": "Detergente Ariel Toque de Downy 2kg + Suavizante Downy Concentrado 1.4L",
    "brand": "Ariel",
    "brandId": 41,
    "brandImageUrl": null,
    "linkText": "detergente-ariel-toque-de-downy-2kg-suavizante-downy-concentrado-1-4l-2",
    "productReference": "998112",
    "productReferenceCode": "998112",
    "categoryId": "1912",
    "productTitle": null,
    "metaTagDescription": null,
    "releaseDate": "2024-08-31T00:00:00Z",
    "clusterHighlights": {
        "323": "Todo Supermercado Metro IO",
        "351": "Todo supermercado 2.0",
        "885": "Delivery gratis todo supermercados por compras may",
        "1081": "Metro 30.10- Detergentes",
        "1421": "Detergente L\u00edquido Concentrado Ariel Doble Poder C",
        "1490": "Coleccion todo Metro",
        "1909": "Detergente en Polvo Ace Lim\u00f3n 4kg + Suavizante Dow",
        "1917": "Coleccion todo Metro II",
        "1953": "Todo supermercado 3",
        "2020": "Todo

In [14]:
# print(json.dumps(products, indent=4))

In [15]:
def print_value(value, key):
    if value:
        print(f"{key} = {value}")
    else:
        print(f"{key} = None")

In [16]:
from bs4 import BeautifulSoup

def clean_html(html_code):
    if html_code is None:
        return ''
    if isinstance(html_code, list):
        html_code = ' '.join(html_code)
    soup = BeautifulSoup(html_code, 'html.parser')
    return soup.get_text()

In [17]:
def get_product_data(product, fecha_db):
    product['Created'] = fecha_db
    new_product = {}
    new_product['productId'] = product.get('productId')
    new_product['productName'] = product.get('productName')
    new_product['brand'] = product.get('brand')
    new_product['brandId'] = product.get('brandId')
    new_product['brandImageUrl'] = product.get('brandImageUrl')
    new_product['categoryId'] = product.get('categoryId')
    new_product['productTitle'] = product.get('productTitle')
    new_product['metaTagDescription'] = product.get('metaTagDescription')
    new_product['categories'] = product.get('categories')
    new_product['categoriesIds'] = product.get('categoriesIds')
    new_product['link'] = product.get('link')
    new_product['Vendido por'] = product.get('Vendido por')
    new_product['Descripción del producto'] = clean_html(product.get('Descripción del producto'))
    new_product['Presentación'] = product.get('Presentación')
    new_product['Presentación/Empaque'] = product.get('Presentación/Empaque')
    new_product['Pack'] = product.get('Pack')
    new_product['Porción Sugerida'] = product.get('Porción Sugerida')
    new_product['Azúcares Por Porción'] = product.get('Azúcares Por Porción')
    new_product['Carbohidratos Por Porción'] = product.get('Carbohidratos Por Porción')
    new_product['Descripción Nutricional'] = product.get('Descripción Nutricional')
    new_product['Grasas Por Porción'] = product.get('Grasas Por Porción')
    new_product['Advertencias de consumo'] = product.get('Advertencias de consumo')
    
    if product['items']:
        new_product['itemId'] = product['items'][0].get('itemId', None)
        new_product['ean'] = product['items'][0].get('ean', None)
        if product['items'][0]['images']:
            new_product['imageUrl'] = product['items'][0]['images'][0].get('imageUrl', None)
            if product['items'][0]['sellers']:
                commertial_offer = product['items'][0]['sellers'][0].get('commertialOffer', {})
                if commertial_offer.get('Installments'):
                    new_product['Value'] = commertial_offer['Installments'][0].get('Value', None)
                new_product['IsAvailable'] = commertial_offer.get('IsAvailable', None)
                new_product['AvailableQuantity'] = commertial_offer.get('AvailableQuantity', None)
                new_product['Price'] = commertial_offer.get('Price', None)
                new_product['ListPrice'] = commertial_offer.get('ListPrice', None)
                new_product['PriceWithoutDiscount'] = commertial_offer.get('PriceWithoutDiscount', None)
                new_product['PriceValidUntil'] = commertial_offer.get('PriceValidUntil', None)

    new_product['Created'] = product['Created']
    
    return new_product

In [ ]:
# Para validar que los datos se están extrayendo correctamente

i = 0
for product in products:
    i += 1
    product['Created'] = fecha_hora_lima_db()
    print_value(product.get('productId'), 'productId')
    print_value(product.get('productName'), 'productName')
    print_value(product.get('brand'), 'brand')
    print_value(product.get('brandId'), 'brandId')
    print_value(product.get('brandImageUrl'), 'brandImageUrl')
    print_value(product.get('categoryId'), 'categoryId')
    print_value(product.get('productTitle'), 'productTitle')
    print_value(product.get('metaTagDescription'), 'metaTagDescription')
    print_value(product.get('categories'), 'categories')
    print_value(product.get('categoriesIds'), 'categoriesIds')
    print_value(product.get('link'), 'link')
    print_value(product.get('Vendido por'), 'Vendido por')
    print_value(clean_html(product.get('Descripción del producto')), 'Descripción del producto')
    print_value(product.get('Presentación'), 'Presentación')
    print_value(product.get('Presentación/Empaque'), 'Presentación/Empaque')
    print_value(product.get('Pack'), 'Pack')
    print_value(product.get('Porción Sugerida'), 'Porción Sugerida')
    print_value(product.get('Azúcares Por Porción'), 'Azúcares Por Porción')
    print_value(product.get('Carbohidratos Por Porción'), 'Carbohidratos Por Porción')
    print_value(product.get('Descripción Nutricional'), 'Descripción Nutricional')
    print_value(product.get('Grasas Por Porción'), 'Grasas Por Porción')
    print_value(product.get('Advertencias de consumo'), 'Advertencias de consumo')
    print_value(product['items'][0]['itemId'], 'itemId')
    print_value(product['items'][0]['ean'], 'ean')
    print_value(product['items'][0]['images'][0]['imageUrl'], 'imageUrl')
    print_value(product['items'][0]['sellers'][0]['commertialOffer']['Installments'][0]['Value'], 'Value')
    print_value(product['items'][0]['sellers'][0]['commertialOffer']['IsAvailable'], 'IsAvailable')
    print_value(product['items'][0]['sellers'][0]['commertialOffer']['AvailableQuantity'], 'AvailableQuantity')
    print_value(product['items'][0]['sellers'][0]['commertialOffer']['Price'], 'Price')
    print_value(product['items'][0]['sellers'][0]['commertialOffer']['ListPrice'], 'ListPrice')
    print_value(product['items'][0]['sellers'][0]['commertialOffer']['PriceWithoutDiscount'], 'PriceWithoutDiscount')
    print_value(product['items'][0]['sellers'][0]['commertialOffer']['PriceValidUntil'], 'PriceValidUntil')
    print_value(product['Created'], 'Created')
    
    if i == 3:
        break

print(len(products))

In [21]:
i=0
for product in products:
    i+=1
    if i % 100 == 0:
        print("Iteración #: ",i)
    new_product = get_product_data(product, fecha_hora_para_base_de_datos)
    df.loc[len(df)] = new_product


Iteración #:  100
Iteración #:  200
Iteración #:  300
Iteración #:  400
Iteración #:  500
Iteración #:  600
Iteración #:  700
Iteración #:  800
Iteración #:  900
Iteración #:  1000
Iteración #:  1100
Iteración #:  1200
Iteración #:  1300
Iteración #:  1400
Iteración #:  1500
Iteración #:  1600
Iteración #:  1700
Iteración #:  1800
Iteración #:  1900
Iteración #:  2000
Iteración #:  2100
Iteración #:  2200
Iteración #:  2300
Iteración #:  2400
Iteración #:  2500
Iteración #:  2600
Iteración #:  2700
Iteración #:  2800
Iteración #:  2900
Iteración #:  3000
Iteración #:  3100
Iteración #:  3200
Iteración #:  3300
Iteración #:  3400
Iteración #:  3500
Iteración #:  3600
Iteración #:  3700
Iteración #:  3800
Iteración #:  3900
Iteración #:  4000
Iteración #:  4100
Iteración #:  4200
Iteración #:  4300
Iteración #:  4400
Iteración #:  4500
Iteración #:  4600
Iteración #:  4700
Iteración #:  4800
Iteración #:  4900
Iteración #:  5000
Iteración #:  5100
Iteración #:  5200
Iteración #:  5300
It

In [22]:
df.sample(n=5)

,productId,productName,brand,brandId,brandImageUrl,categoryId,productTitle,metaTagDescription,categories,categoriesIds,...,ean,imageUrl,Value,IsAvailable,AvailableQuantity,Price,ListPrice,PriceWithoutDiscount,PriceValidUntil,Created
7751,9468,Shampoo Head & Shoulders Suave y Manejable 2 e...,Head & Shoulders,403,None,2212,Shampoo Head & Shoulders Suave y Manejable 2 e...,¡Encuentra aquí los mejores productos de Shamp...,"[/Higiene, Salud y Belleza/Cuidado del Cabello...","[/2200/2202/2212/, /2200/2202/, /2200/]",...,7500435019828,https://metroio.vteximg.com.br/arquivos/ids/25...,17.90,True,99999,17.90,19.5,17.9,2100-01-01T00:00:00Z,2024-10-14 19:21:54
32976,1009817,Petit Pan Metro 25un,Metro,2000891,None,1001655,Petit Pan Metro 25un,¡Encuentra aquí los mejores productos de Panes...,[/Panadería y Pastelería/La Panadería/Panes Em...,"[/1001374/1001375/1001655/, /1001374/1001375/,...",...,2050043860305,https://metroio.vteximg.com.br/arquivos/ids/52...,7.90,True,99999,7.90,7.9,7.9,2100-01-01T00:00:00Z,2024-10-14 19:21:54
27741,1033058,Mug Jumbo Plus Stitch 390ml,Stitch,2004322,None,1000790,Mug Jumbo Plus Stitch 390ml,¡Encuentra aquí la mejor variedad en productos...,"[/Hogar y Bazar/Menaje de Mesa/Jarras, Garrafa...","[/1000131/1000241/1000790/, /1000131/1000241/,...",...,8412497758692,https://metroio.vteximg.com.br/arquivos/ids/51...,21.90,True,1,21.90,21.9,21.9,2100-01-01T00:00:00Z,2024-10-14 19:21:54
830,945405,Paleta de Helado Cuisine & Co Fresa con Leche ...,Cuisine & Co,9885,None,1000760,Paleta de Helado Cuisine & Co Fresa con Leche ...,None,"[/Congelados/Helados/Helados Nacionales/, /Con...","[/1200/1203/1000760/, /1200/1203/, /1200/]",...,2050043848341,https://metroio.vteximg.com.br/arquivos/ids/36...,7.11,True,100,7.11,7.9,7.9,2100-01-01T00:00:00Z,2024-10-14 19:21:54
31604,1002126,Pan Integral Bimbo Cero Cero 560g,Bimbo,76,None,1001271,Pan Integral Bimbo Cero Cero 560g,¡Encuentra aquí los mejores productos de Panes...,[/Desayuno/Panes y Tortillas Empacadas/Panes E...,"[/1001253/1001270/1001271/, /1001253/1001270/,...",...,7750727012666,https://metroio.vteximg.com.br/arquivos/ids/40...,12.20,True,100,12.20,12.2,12.2,2100-01-01T00:00:00Z,2024-10-14 19:21:54


In [23]:
df[['productId', 'productName', 'brand', 'categoryId', 'link', 'Price', 'AvailableQuantity']].sample(n=8)

,productId,productName,brand,categoryId,link,Price,AvailableQuantity
17898,12204,Detergente en Polvo Ace Pétalos Florales 2kg,Ace,1912,https://www.metro.pe/detergente-en-polvo-ace-p...,29.90,100
10704,946759,Oster Cafetera de Filtro BVSTDCP12B 12 Tazas,Oster,1000011,https://www.metro.pe/oster-cafetera-de-filtro-...,189.00,100
12902,1028349,Botella Basa Cantimplora 800ml,Basa,1001616,https://www.metro.pe/botella-basa-cantimplora-...,24.90,10
20837,4350,Cerveza Artesanal Sierra Andina Pachacutec Bot...,Sierra Andina,1000707,https://www.metro.pe/cerveza-artesanal-sierra-...,11.90,100
24303,1002750,LAPIZ GRAFITO TITAN 2B NEGRO X 12,Faber Castell,1000883,https://www.metro.pe/lapiz-grafito-titan-2b-ne...,6.50,100
19799,1044450,Baileys Irish Cream 750ml + Cápsulas de Café D...,Baileys,1001667,https://www.metro.pe/baileys-irish-cream-750ml...,99.90,99999
1062,1014175,Morcilla 300 g,Casa Europa,1406,https://www.metro.pe/morcilla-300-g/p,15.40,10
32745,58468,Torta Selva Negra 10 Porciones,Cuisine & Co,1001384,https://www.metro.pe/torta-selva-negra-10-porc...,42.21,10


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 35281 entries, 0 to 35280
Data columns (total 33 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   productId                  35281 non-null  object 
 1   productName                35281 non-null  object 
 2   brand                      35281 non-null  object 
 3   brandId                    35281 non-null  int64  
 4   brandImageUrl              0 non-null      object 
 5   categoryId                 35281 non-null  object 
 6   productTitle               27547 non-null  object 
 7   metaTagDescription         22382 non-null  object 
 8   categories                 35281 non-null  object 
 9   categoriesIds              35281 non-null  object 
 10  link                       35281 non-null  object 
 11  Vendido por                0 non-null      object 
 12  Descripción del producto   35281 non-null  object 
 13  Presentación               270 non-null    object 


In [25]:
df['product_uri_json'] = df['productId'].apply(lambda x: f'https://www.metro.pe/api/catalog_system/pub/products/search?fq=productId:{x}')

In [26]:
df['product_uri_json'].sample(n=5)

11983    https://www.metro.pe/api/catalog_system/pub/pr...
28730    https://www.metro.pe/api/catalog_system/pub/pr...
20115    https://www.metro.pe/api/catalog_system/pub/pr...
29307    https://www.metro.pe/api/catalog_system/pub/pr...
33077    https://www.metro.pe/api/catalog_system/pub/pr...
Name: product_uri_json, dtype: object

In [27]:
file_name_productos_csv = f'productos_{fecha_hora_para_archivos}.csv'
df.to_csv(file_name_productos_csv, index=False)

In [28]:
df.iloc[26955]

productId                                                               934860
productName                                Mesa M+Design con Bandeja Negro 2.0
brand                                                                 M+DESIGN
brandId                                                                2001579
brandImageUrl                                                             None
categoryId                                                             1000806
productTitle                                                              None
metaTagDescription                                                        None
categories                   [/Hogar y Bazar/Muebles/Muebles Multiusos/, /H...
categoriesIds                [/1000131/1000187/1000806/, /1000131/1000187/,...
link                         https://www.metro.pe/mesa-m-design-con-bandeja...
Vendido por                                                               None
Descripción del producto                            